![alt text](https://drive.google.com/uc?export=view&id=1UXScsVx_Wni_JuDdB8LeTnM6jsPfIwkW)

Proprietary content. © Great Learning. All Rights Reserved. Unauthorized use or distribution prohibited.

# Sentiment Classification

### Dataset
- Dataset of 50,000 movie reviews from IMDB, labeled by sentiment positive (1) or negative (0)
- Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers).
- For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".
- As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.

Command to import data
- `from tensorflow.keras.datasets import imdb`

### Import the data (2 Marks)
- Use `imdb.load_data()` method
- Get train and test set
- Take 10000 most frequent words

In [ ]:
from tensorflow.keras.datasets import imdb

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000, maxlen=300)

In [ ]:
x_train_max_len = max(len(x) for x in x_train)
print("Max length of sequence in x_train: {}".format(x_train_max_len))

x_test_max_len = max(len(x) for x in x_test)
print("Max length of sequence in x_test:  {}".format(x_test_max_len))

### Pad each sentence to be of same length (2 Marks)
- Take maximum sequence length as 300

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train_padded = pad_sequences(x_train, maxlen=300)
x_test_padded = pad_sequences(x_test, maxlen=300)

### Print shape of features & labels (2 Marks)

Number of review, number of words in each review

In [ ]:
print("Number of reviews in x_train: {}".format(len(x_train)))
print("Number of reviews in x_test: {}".format(len(x_train)))

In [ ]:
print("Number of words in each review is {0}".format(x_train_padded[0].shape[0]))

Number of labels

In [ ]:
import numpy as np

print("Labels: {}".format(np.unique(y_train)))

In [ ]:
print("Number of labels: {}".format(len(np.unique(y_train))))

### Print value of any one feature and it's label (2 Marks)

Feature value

In [ ]:
print("Movie Review:")
print(x_train[0])

Label value

In [ ]:
print("Sentiment: {}".format(y_train[0]))

### Decode the feature value to get original sentence (2 Marks)

First, retrieve a dictionary that contains mapping of words to their index in the IMDB dataset

In [ ]:
word_to_index = imdb.get_word_index()
word_to_index = {k:(v+3) for k,v in word_to_index.items()}

word_to_index["<PAD>"] = 0
word_to_index["<START>"] = 1
word_to_index["<UNK>"] = 2

index_to_word = {value:key for key,value in word_to_index.items()}

Now use the dictionary to get the original words from the encodings, for a particular sentence

In [ ]:
decoded = " ".join([index_to_word[i] for i in x_train[2]])
print(decoded)

Get the sentiment for the above sentence
- positive (1)
- negative (0)

In [ ]:
y_train[2]

### Define model (10 Marks)
- Define a Sequential Model
- Add Embedding layer
  - Embedding layer turns positive integers into dense vectors of fixed size
  - `tensorflow.keras` embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unique integer number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn LabelEncoder.
  - Size of the vocabulary will be 10000
  - Give dimension of the dense embedding as 100
  - Length of input sequences should be 300
- Add LSTM layer
  - Pass value in `return_sequences` as True
- Add a `TimeDistributed` layer with 100 Dense neurons
- Add Flatten layer
- Add Dense layer

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, TimeDistributed, Flatten, Dense

model = Sequential()
model.add(Embedding(10000, 100, input_length=300))
model.add(LSTM(100, return_sequences=True))
dense_layer = Dense(100, activation='relu')
model.add(TimeDistributed(dense_layer))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

### Compile the model (2 Marks)
- Use Optimizer as Adam
- Use Binary Crossentropy as loss
- Use Accuracy as metrics

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Print model summary (2 Marks)

In [ ]:
model.summary()

### Fit the model (2 Marks)

In [ ]:
model.fit(x_train_padded, y_train, epochs=5, batch_size=256)

### Evaluate model (2 Marks)

In [ ]:
score, acc = model.evaluate(x_test_padded, y_test, batch_size=512)
print("Score: {:.4f}".format(score))
print("Accuracy: {:.4f}".format(acc))

### Predict on one sample (2 Marks)

In [ ]:
good = "it was great movie i had fun story line was awesome"
bad = "worst movie i have ever seen"

In [ ]:
for review in [good, bad]:
    encoded_review = []
    review_split = review.split(" ")
    for word in review_split:
        encoded_review.append(word_to_index[word])
    review_padded = pad_sequences([encoded_review], maxlen=300)
    pred = model.predict(review_padded)
    if pred > 0.5:
        sentiment = 'positive'
    else:
        sentiment = 'negative'
    print("Review: {0}\n\tSentiment: {1}".format(review, sentiment))